In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor

df = pd.read_csv('data_encoded.csv')

X = df.drop('overall_rating', axis=1)
y = df['overall_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [2]:
# Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
base_learners = [
    ('svr', SVR()),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('knn', KNeighborsRegressor()),
    ('rf', RandomForestRegressor(random_state=42)),
    ('ada', AdaBoostRegressor(random_state=42)),
    ('gb', GradientBoostingRegressor(random_state=42)),
    ('xgb', XGBRegressor(random_state=42)),
    ('mlp', MLPRegressor(random_state=42, max_iter=500))
]

In [9]:
meta_learner = LinearRegression()

stack = StackingRegressor(
    estimators=base_learners, 
    final_estimator=meta_learner
)

stack.fit(X_train_scaled, y_train)

print(stack.score(X_test_scaled, y_test))

0.9384775893056342


In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
def evaluate_model(model, X, y):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    return mse, mae, r2

def print_metrics(mse, mae, r2):
    print(f'MSE: {mse:.3f}')
    print(f'MAE: {mae:.3f}')
    print(f'R2: {r2:.3f}')

In [11]:
y_train_pred = stack.predict(X_train_scaled)

print('Train Set:')
print_metrics(*evaluate_model(stack, X_train_scaled, y_train))

Train Set:
MSE: 1.495
MAE: 0.706
R2: 0.969


In [12]:
y_pred = stack.predict(X_test_scaled)

print('Test Set:')
print_metrics(*evaluate_model(stack, X_test_scaled, y_test))

Test Set:
MSE: 2.916
MAE: 0.998
R2: 0.938
